# Análisis exploratorio de datos

El plan es no eliminar outliers en esta fase del proyecto, pero si ajustar los límites de los gráficos para que se vean más claros

In [ ]:
# paquetes
import pandas as pd # para dataframes
import seaborn as sns # para gráficas cool
import matplotlib.pyplot as plt # para ajustes de gráficas sns
import statistics # para estadísticos
import numpy as np # para operaciones numéricas
from sklearn.decomposition import PCA # para el PCA
from sklearn.preprocessing import StandardScaler # para estandarizar datos

# ajustes del output de pandas
pd.options.display.max_columns = 100 # cuántas columnas se imprimen

Para ejecutar este código, es necesario primero ejecutar el markdown "Obtencion_Datos.ipynb".

In [ ]:
# importación de los datos
datos = pd.read_csv('df_datos.csv', index_col='Unnamed: 0')

# un resumen rápido de la información del df
datos.info()

## Boxplots

Los boxplots de seaborn muestran la siguiente información
- mediana (línea de en medio).
- Q1 y Q3 (límites de la caja).
- Q1 - 1.5*IQR (límite del bigote inferior...).
- Q3 + 1.5*IQR (... y superior, donde IQR = InterQuartile Ratio = Q3-Q1).
- Outliers (datos fuera de los bigotes).

In [ ]:
# creamos la figura
fig, axs = plt.subplots(4,4, figsize=(15, 15))

# agregamos los gráficos uno por uno
z = 0 # para obtener solo una vez la info de leyendas
# flatten nos va a ayudar a ordenar unidimensionalmente la ubicación de los axis
# zip nos va a ayudar a regresar en pares ordenados la info de col,ubi
for columna, ubicacion in zip(datos.columns[2:-5],axs.flatten()):
    #sacamos Q1,Q2 y el rango intercuartilico para definir los límites del eje x
    Q1,Q3 = np.quantile(datos[columna].dropna(),[0.25,0.75])
    RIQ = Q3-Q1
    li = Q1 - 2*RIQ
    ls = Q3 + 3.5*RIQ
    sns.boxplot(datos, x=columna, hue='categoria', ax=ubicacion)
    ubicacion.set_xlim([li,ls])
    ubicacion.get_legend().remove() # quitamos las leyendas individuales de los gráficos
    if z == 0: # para el primer ciclo...
        handles, labels = ubicacion.get_legend_handles_labels() # obtenemos la info de leyendas
        z =+ 1 # cambiamos z para no volver a sacar la info
# agregamos la leyenda global
fig.legend(handles, labels, loc='upper center', ncols = 4, bbox_to_anchor=(0.5, 0.92))
plt.show() # imprimimos la imagen

## Correlación de las variables contínuas

In [ ]:
# matríz de correlación
fig, ax = plt.subplots(figsize=(10,10))
datos_correlacion = datos[datos.columns[2:-5]].corr()
sns.heatmap(datos_correlacion,
            annot=True,
            fmt=".1f",
            cmap=sns.color_palette("vlag",as_cmap=True),
            xticklabels=datos_correlacion.columns.values,
            yticklabels=datos_correlacion.columns.values)
plt.show()

In [ ]:
# representación gráfica de las correlaciones
sns.pairplot(data=datos, hue='categoria', vars=datos.columns[2:-5])
plt.show()

## Análisis de Componentes Principales

In [ ]:
# PCA

# primero arreglamos los datos
datos_pca_originales = datos.drop(['id_ecg','categoria','patient_id','sex'],axis=1).dropna() # quitamos algunas columnas no numéricas y los NA.
scaler = StandardScaler() # llamamos al normalizador
datos_pca = scaler.fit_transform(datos_pca_originales) # normalizamos

# aplicamos PCA
pca = PCA(n_components=2) # llamamos al PCAdor
datos_pca = pca.fit_transform(datos_pca) # aplicamos PCA

# calculamos los pesos de cada variable en cada componente (se usan abajo)
componentes = pca.components_  # Matriz de pesos: cada fila es un componente (PC1, PC2, ...)
componentes = pd.DataFrame(componentes, columns=datos_pca_originales.columns) # transformamos los valores en un df
print(f"\nCargas de las variables en los componentes principales:\n{componentes}")

In [ ]:
# scatterplot del PCA
plt.figure(figsize=(8, 6))        # tamaño del gráfico
plt.scatter(x = datos_pca[:, 0],  # eje x será el PCA1
            y = datos_pca[:, 1],  # eje y será el PCA2
            c='blue',                 # color de los puntitos
            edgecolor='k',            # color del perímetro de los puntitos. k es negro
            s=60)                     # tamaño de los puntitos             
plt.title('Proyección de los Datos en el Espacio de los Componentes Principales')
plt.xlabel('Primer Componente Principal (PC1)')
plt.ylabel('Segundo Componente Principal (PC2)')
plt.grid()
plt.show()

In [ ]:
# gráfico de barras con el peso de cada variable en cada componente principal
plt.figure(figsize=(8, 6))
componentes.T.plot(kind='bar', figsize=(10, 6), legend=True)
plt.title('Peso de cada variable en los Componentes Principales (PC1, PC2)')
plt.ylabel('Cargas (Pesos)')
plt.xlabel('Variables')
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.show()

## Edad y Sexo

In [ ]:
# calculamos los datos centrales
subdatos = datos[~datos[['age','sex']].isna().any(axis=1)][['age','sex']] # quitamos los NA
minimo = min(subdatos['age'])
cinco_porciento = np.percentile(subdatos['age'],5)
maximo = max(subdatos['age'])
noventaycinco_porciento = np.percentile(subdatos['age'],95)
media = statistics.mean(subdatos['age'])
[hombres, mujeres] = subdatos["sex"].value_counts(ascending=True)

#imprimimos
print("EDAD")
print("mínimo: " + str(minimo))
print("percentil 5%: " + str(cinco_porciento))
print("promedio: " + str(media))
print("percentil 95%: " + str(noventaycinco_porciento))
print("máximo: " + str(maximo))
print("")
print("SEXO")
print("mujeres: " + str(mujeres))
print("porcentaje mujeres: " + str(mujeres/(mujeres+hombres)))
print("hombres: " + str(hombres))
print("porcentaje hombres: " + str(hombres/(mujeres+hombres)))

In [ ]:
#histograma edad
sns.histplot(subdatos[subdatos['age'] < 100]['age'], element="step", bins=20)
plt.show()

In [ ]:
#histograma sexo
sns.barplot(x = ["mujeres", "hombres"], y = [mujeres, hombres])
plt.show()